# SSD Object Detecion with Tensorflow Object Detection API

In [1]:
# Prerequisites and Dependencies:
import os
from PIL import Image
import numpy as np
import tarfile
import urllib.request

import tensorflow as tf

# Dependencies
# !pip install tensorflow
# !pip install tf_slim 
# !pip install tensorflow-object-detection-api

### Download Pre-Trained Models form Tensorflow Object Detection Library

In [2]:
MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
BASE_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_URL = f'{BASE_URL}/{MODEL_FILE}'

# Download and extract model
urllib.request.urlretrieve(DOWNLOAD_URL, MODEL_FILE)
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

# Path to the saved model directory
MODEL_DIR = os.path.join(os.getcwd(), MODEL_NAME, 'saved_model')

C:\Users\Jari\AppData\Local\Temp\ipykernel_109492\314816498.py:9: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall()


### Load the Model

In [3]:
# Load the Saved Model
detect_fn = tf.saved_model.load(MODEL_DIR)
print("Model loaded successfully")

Model loaded successfully


### Load an Image

In [4]:
def load_image_into_numpy_array(path):
    return np.array(Image.open(path))

IMAGE_PATH = '../images/traffic.jpg' 
image_np = load_image_into_numpy_array(IMAGE_PATH)

# Add batch dimension and convert to tensor
input_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]

### Run Inference

In [5]:
detections = detect_fn(input_tensor)

# Convert outputs to NumPy for easier handling
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# Detection classes are ints
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

print("Number of objects detected:", num_detections)

Number of objects detected: 100


### Visualize

In [ ]:
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import label_map_util
import matplotlib.pyplot as plt

# Download label map for COCO dataset (if not already)
LABEL_FILENAME = 'mscoco_label_map.pbtxt'
urllib.request.urlretrieve(
    'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt',
    LABEL_FILENAME
)

category_index = label_map_util.create_category_index_from_labelmap(LABEL_FILENAME)

viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np,
    detections['detection_boxes'],
    detections['detection_classes'],
    detections['detection_scores'],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=10,
    min_score_thresh=0.5
)

plt.figure(figsize=(12,8))
plt.imshow(image_np)
plt.axis('off')
plt.show()